In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [7]:
ttnc_train = pd.read_csv('train.csv', index_col='Name')
# ttnc_train
ttnc_test = pd.read_csv('test.csv',index_col='Name')
# ttnc_test
ttnc_gender = pd.read_csv('gender_submission.csv')
# ttnc_gender
train_df = pd.DataFrame(ttnc_train)
test_df = pd.DataFrame(ttnc_test)
train_df.head()
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Name,,,,,,,,,,
"Kelly, Mr. James",892,3,male,34.5,0,0,330911,7.8292,NaN,Q
"Wilkes, Mrs. James (Ellen Needs)",893,3,female,47.0,1,0,363272,7.0000,NaN,S
"Myles, Mr. Thomas Francis",894,2,male,62.0,0,0,240276,9.6875,NaN,Q
"Wirz, Mr. Albert",895,3,male,27.0,0,0,315154,8.6625,NaN,S
"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",896,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
desc = test_df.isnull()
desc.tail()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Name,,,,,,,,,,
"Spector, Mr. Woolf",False,False,False,True,False,False,False,False,True,False
"Oliva y Ocana, Dona. Fermina",False,False,False,False,False,False,False,False,False,False
"Saether, Mr. Simon Sivertsen",False,False,False,False,False,False,False,False,True,False
"Ware, Mr. Frederick",False,False,False,True,False,False,False,False,True,False
"Peter, Master. Michael J",False,False,False,True,False,False,False,False,True,False


In [9]:
labelencoder = LabelEncoder()

In [10]:
test_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, Kelly, Mr. James to Peter, Master. Michael J
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          332 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Ticket       418 non-null    object 
 7   Fare         417 non-null    float64
 8   Cabin        91 non-null     object 
 9   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 35.9+ KB


In [11]:
sex = labelencoder.fit_transform(test_df['Sex'])
ticket = labelencoder.fit_transform(test_df['Ticket'])

In [12]:
test_df['Sex'] = sex
test_df['Ticket'] = ticket

In [13]:
test_df.drop(['Cabin','Embarked'], axis=1, inplace=True)

In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, Kelly, Mr. James to Peter, Master. Michael J
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    int32  
 3   Age          332 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Ticket       418 non-null    int32  
 7   Fare         417 non-null    float64
dtypes: float64(2), int32(2), int64(4)
memory usage: 26.1+ KB


In [15]:
test_df.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,0.636364,30.272590,0.447368,0.392344,180.944976,35.627188
std,120.810458,0.841838,0.481622,14.181209,0.896760,0.981429,107.533763,55.907576
min,892.000000,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,0.000000,21.000000,0.000000,0.000000,85.250000,7.895800
50%,1100.500000,3.000000,1.000000,27.000000,0.000000,0.000000,181.000000,14.454200
75%,1204.750000,3.000000,1.000000,39.000000,1.000000,0.000000,279.750000,31.500000
max,1309.000000,3.000000,1.000000,76.000000,8.000000,9.000000,362.000000,512.329200


In [16]:
test_df.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Ticket          0
Fare            1
dtype: int64

In [17]:
age = test_df['Age']
age_miss = age.mean()
age_miss

30.272590361445783

In [18]:
age_miss = int(age_miss)
age.fillna(age_miss,inplace=True)
age.describe()

count    418.000000
mean      30.216507
std       12.635016
min        0.170000
25%       23.000000
50%       30.000000
75%       35.750000
max       76.000000
Name: Age, dtype: float64

In [26]:
x = train_df.drop('Survived',axis=1)
y = train_df['Survived'].values
x
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,